In [1]:
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
train = np.load('train.npy')
test = np.load('test.npy')

In [3]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train[:,0]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train[:,1]])
angle=train[:,3]
target_train = train[:,4]

In [4]:
np.count_nonzero(angle)

1604

In [5]:
angle[angle=='na']=0

In [6]:
angle

array([43.9239, 38.1562, 45.2859, ..., 0, 0, 0], dtype=object)

In [7]:
np.count_nonzero(angle)

1471

In [8]:
np.sum(angle)/np.count_nonzero(angle)

39.26870747790618

In [9]:
angle[angle == 0] = np.sum(angle)/np.count_nonzero(angle)

In [10]:
for i in range(len(angle)):
    angle[i]=np.full((75,75),angle[i])

In [11]:
X_band_3 = np.empty([len(angle),75,75],dtype=np.float32)

for i in range(len(angle)):
    for j in range(75):
        for k in range(75):
            X_band_3[i][j][k] = (X_band_1[i][j][k]-X_band_2[i][j][k])/angle[i][j][k]

In [12]:
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis,]/30, X_band_2[:, :, :, np.newaxis,]/30,X_band_3[:, :, :, np.newaxis,]], axis=-1)

In [17]:
np.save('X_train',X_train)

In [13]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, BatchNormalization
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Using TensorFlow backend.
/Users/yaqiongzhang/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [14]:
#define our model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(BatchNormalization())
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

file_path = "model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

In [16]:
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, train_size=0.9)
#Without denoising, core features.

import os
gmodel=getModel()
history = gmodel.fit(X_train_cv, y_train_cv,
          batch_size=24,
          epochs=30,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
batch_normalization_5 (Batch (None, 73, 73, 64)        256       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
batch_normalization_6 (Batch (None, 71, 71, 64)        256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 33, 33, 64)        36928     
__________